# Data Augmentation using Keyword to Sentence Technique

## Importing the Librariers

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from .base.BaseKeywordExtractor import BaseKeywordExtractor

In [5]:
# import spacy
import tqdm as notebook_tqdm
from collections import Counter
from string import punctuation

In [6]:
# !export http_proxy=http://sysproxy.wal-mart.com:8080
# !export https_proxy=http://sysproxy.wal-mart.com:8080
# !python -m spacy download en
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_lg
# nlp = spacy.load("en_core_web_sm")
# !python -m spacy download en

## Reading the Data

In [7]:
# import chardet

# with open('spam.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(100000))

# print(result)

In [8]:
df = pd.read_csv("spam.csv", encoding='latin1')
df=df[["v1","v2"]]
print(df.shape)
df.head(1)

(5572, 2)


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."


In [9]:
df=df.drop_duplicates()
print(df.shape)
df.head(1)

(5169, 2)


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."


In [10]:
df.columns=["Line_item_target","line_item_narative"]
print(df.shape)
df.head()

(5169, 2)


,Line_item_target,line_item_narative
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df["Line_item_target"].value_counts(normalize=True)

ham     0.87367
spam    0.12633
Name: Line_item_target, dtype: float64

## Creating the Target Var

In [12]:
df["label"]=np.where(df["Line_item_target"]=="spam",1,0)

In [13]:
df["label"].value_counts()

0    4516
1     653
Name: label, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame and "target" is your target variable
X = df["line_item_narative"]  # Features
y = df["label"]  # Target variable

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# If you want to keep track of the train/test in your dataframe you can do something like:
df["train_test_tag"] = "train"
df.loc[X_test.index, "train_test_tag"] = "test"

In [15]:
pd.crosstab(df["train_test_tag"],df["label"],normalize='index',margins=True)

label,0,1
train_test_tag,,
test,0.862837,0.137163
train,0.879007,0.120993
All,0.873670,0.126330


In [16]:
df["train_test_tag"].value_counts()

train    3463
test     1706
Name: train_test_tag, dtype: int64

## Ranking Popular/Least Popular Keywords

In [17]:
# word count are min 1 to max 175 
# Identify most common words
common = pd.Series(' '.join(df['line_item_narative']).split()).value_counts()[:410]
common

to         1972
you        1562
I          1375
a          1198
the        1114
           ... 
home.        26
looking      26
says         25
hour         25
Wat          25
Length: 410, dtype: int64

#### I found the most common words were the most useless ones like: the, and, of a, in, is, to, this, with, from and for. These are called syncategorematic words and do not make sense on their own and therefore checked against the stop list. If it didn’t contain them then they should be added. Then I checked the uncommon words.
    

In [18]:
#Identify uncommon words
uncommon =  pd.Series(' '.join(df 
         ['line_item_narative']).split()).value_counts()[-10:]
uncommon

YOU,CLEAN     1
tag           1
BLOOD.Send    1
Ten           1
it,pls,pls    1
annoying      1
consensus     1
entertain     1
laugh,        1
Rofl.         1
dtype: int64

## Cleaning the data

In [19]:
# convert to lowercase
df['description'] = df['line_item_narative'].str.lower()
# dataset['title'] = dataset['title'].str.lower()

# convert desc to string
df['description'] = df['description'].astype(str)
df.dtypes

Line_item_target      object
line_item_narative    object
label                  int64
train_test_tag        object
description           object
dtype: object

### Creating the stop word list and adding to it

In [21]:
# library for text preprocessing
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/deepaksingla/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
# create a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))

# create a list of custom stopwords
new_words =  ['the', 'a ','us', 'sold', 'of', 'in', 'created', 'that', 'made', 'we\'ve', 'after','their','his','only']
stop_words = stop_words.union(new_words)
stop_words

{'a',
 'a ',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'created',
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'made',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',


### Removing useless words from the DataFrame column

In [25]:
df['description'] = df['description'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

df.head(5)


,Line_item_target,line_item_narative,label,train_test_tag,description
0,ham,"Go until jurong point, crazy.. Available only ...",0,train,"go jurong point, crazy.. available bugis n gre..."
1,ham,Ok lar... Joking wif u oni...,0,train,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,train,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,0,train,u dun say early hor... u c already say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,train,"nah think goes usf, lives around though"


In [26]:
print(df["line_item_narative"].iloc[0])
print(df["description"].iloc[0])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...


### Adding keyword count

In [27]:
# Fetch wordcount for each description
df['word_count'] = df['description'].apply(lambda x: len(str(x).split(" ")))
df[['description','word_count']].head(5)

,description,word_count
0,"go jurong point, crazy.. available bugis n gre...",16
1,ok lar... joking wif u oni...,6
2,free entry 2 wkly comp win fa cup final tkts 2...,23
3,u dun say early hor... u c already say...,9
4,"nah think goes usf, lives around though",7


In [28]:
df_major=df[df["label"]==0]
df_minor=df[df["label"]==1]
print(df.shape)
print(df_major.shape)
print(df_minor.shape)

(5169, 6)
(4516, 6)
(653, 6)


## Extracting key words

### Yake

In [30]:
import yake

In [31]:
pd.set_option('max_colwidth', 400)

In [32]:
kw_extractor = yake.KeywordExtractor()

In [33]:
def kw_extractor_cutoff(text,cutoff):
    ls=[]
    for i in range(len(text)):
        if text[i][1]>cutoff:
            ls.append(text[i][0])
        else :
            ls=ls
    return ls

In [34]:
df["yake_keywords"]=df["description"].apply(lambda x:kw_extractor.extract_keywords(x))

In [35]:
df["yake_keywords_cutoff"]=df["yake_keywords"].apply(lambda x:kw_extractor_cutoff(x,0.2))

In [36]:
print(df.shape)
df.head(1)

(5169, 8)


,Line_item_target,line_item_narative,label,train_test_tag,description,word_count,yake_keywords,yake_keywords_cutoff
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0,train,"go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...",16,"[(crazy., 0.06588837669267192), (jurong point, 0.0934346755383478), (point, 0.2200560909321288), (jurong, 0.38831292435681075), (buffet, 0.4949246952252326), (cine, 0.5880798524606783), (wat, 0.5880798524606783), (bugis, 0.68796622281612), (great, 0.68796622281612), (world, 0.68796622281612), (amore, 0.7625957055371981), (cine got amore, 0.8131286944405928), (amore wat, 0.8131286944405928), (b...","[point, jurong, buffet, cine, wat, bugis, great, world, amore, cine got amore, amore wat, bugis n great, great world]"


In [37]:
df["yake_keywords_cutoff_ud"]=df["yake_keywords_cutoff"].apply(lambda x:','.join(x))

In [38]:
df[["description","yake_keywords","yake_keywords_cutoff","yake_keywords_cutoff_ud"]].head(5)

,description,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud
0,"go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...","[(crazy., 0.06588837669267192), (jurong point, 0.0934346755383478), (point, 0.2200560909321288), (jurong, 0.38831292435681075), (buffet, 0.4949246952252326), (cine, 0.5880798524606783), (wat, 0.5880798524606783), (bugis, 0.68796622281612), (great, 0.68796622281612), (world, 0.68796622281612), (amore, 0.7625957055371981), (cine got amore, 0.8131286944405928), (amore wat, 0.8131286944405928), (b...","[point, jurong, buffet, cine, wat, bugis, great, world, amore, cine got amore, amore wat, bugis n great, great world]","point,jurong,buffet,cine,wat,bugis,great,world,amore,cine got amore,amore wat,bugis n great,great world"
1,ok lar... joking wif u oni...,"[(lar, 0.2005079697193566), (joking wif, 0.4456055016437946), (wif u oni, 0.4456055016437946), (joking, 0.46553351027698087), (oni, 0.46553351027698087), (wif, 0.6621399683957523)]","[lar, joking wif, wif u oni, joking, oni, wif]","lar,joking wif,wif u oni,joking,oni,wif"
2,free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's,"[(wkly comp win, 0.008084401239461886), (cup final tkts, 0.008084401239461886), (wkly comp, 0.03313312481282493), (final tkts, 0.03313312481282493), (comp win, 0.054142070592351293), (win fa cup, 0.054142070592351293), (cup final, 0.054142070592351293), (free entry, 0.06661172096316262), (receive entry question, 0.08821275592953637), (std txt rate, 0.1054662887875882), (free, 0.141510365575810...","[entry, std txt, txt rate, comp, win]","entry,std txt,txt rate,comp,win"
3,u dun say early hor... u c already say...,"[(early hor, 0.02541909697430123), (dun say early, 0.04188746641162499), (hor, 0.12363091320521931), (dun, 0.2005079697193566), (early, 0.2005079697193566)]","[dun, early]","dun,early"
4,"nah think goes usf, lives around though","[(nah, 0.15831692877998726), (usf, 0.15831692877998726), (lives, 0.15831692877998726)]",[],


In [45]:
## Writing the Data
df.to_csv("sample_kw_spam.csv",index=False)

## Using GPTNeo to make sentence from keys

In [1]:
from happytransformer import HappyGeneration, GENSettings
from transformers import pipeline

: 

: 

In [ ]:
df=pd.read_csv("sample_kw_spam.csv")
print(df.shape)
df.head(1)

(5572, 9)


,Line_item_target,line_item_narative,label,train_test_tag,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0,train,"go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...",16,"[('crazy.', 0.06588837669267192), ('jurong point', 0.0934346755383478), ('point', 0.2200560909321288), ('jurong', 0.38831292435681075), ('buffet', 0.4949246952252326), ('cine', 0.5880798524606783), ('wat', 0.5880798524606783), ('bugis', 0.68796622281612), ('great', 0.68796622281612), ('world', 0.68796622281612), ('amore', 0.7625957055371981), ('cine got amore', 0.8131286944405928), ('amore wat...","['point', 'jurong', 'buffet', 'cine', 'wat', 'bugis', 'great', 'world', 'amore', 'cine got amore', 'amore wat', 'bugis n great', 'great world']","point,jurong,buffet,cine,wat,bugis,great,world,amore,cine got amore,amore wat,bugis n great,great world"


In [58]:
df["label"].value_counts()

label
0    4825
1     747
Name: count, dtype: int64

In [55]:
df_train=df[df["train_test_tag"]=="train"]
print(df_train.shape)
df_train.head(1)

(3900, 9)


,Line_item_target,line_item_narative,label,train_test_tag,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0,train,"go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...",16,"[('crazy.', 0.06588837669267192), ('jurong point', 0.0934346755383478), ('point', 0.2200560909321288), ('jurong', 0.38831292435681075), ('buffet', 0.4949246952252326), ('cine', 0.5880798524606783), ('wat', 0.5880798524606783), ('bugis', 0.68796622281612), ('great', 0.68796622281612), ('world', 0.68796622281612), ('amore', 0.7625957055371981), ('cine got amore', 0.8131286944405928), ('amore wat...","['point', 'jurong', 'buffet', 'cine', 'wat', 'bugis', 'great', 'world', 'amore', 'cine got amore', 'amore wat', 'bugis n great', 'great world']","point,jurong,buffet,cine,wat,bugis,great,world,amore,cine got amore,amore wat,bugis n great,great world"


In [64]:
generator_bn = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [65]:
generator_mn = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [66]:
generator2 = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [68]:
# generator("subpoena,duces,tecum", max_length=30, do_sample=True, temperature=0.9)

In [69]:
# generator("application", max_length=30, do_sample=True, temperature=0.9)

In [70]:
# df["yake_keywords_cutoff_ud"].values

In [71]:
# print(df.shape)
# df.head(1)

## UsinG Happy gen to generate text

In [77]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [56]:
def gen_text_fn(model,text_data):
    result=model.generate_text(text_data)
    return result.text

In [57]:
df_reject=df_train[df_train["label"]==1]
print(df_reject.shape)
df_reject.head(1)

(528, 9)


,Line_item_target,line_item_narative,label,train_test_tag,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1,train,free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's,23,"[('wkly comp win', 0.008084401239461886), ('cup final tkts', 0.008084401239461886), ('wkly comp', 0.03313312481282493), ('final tkts', 0.03313312481282493), ('comp win', 0.054142070592351293), ('win fa cup', 0.054142070592351293), ('cup final', 0.054142070592351293), ('free entry', 0.06661172096316262), ('receive entry question', 0.08821275592953637), ('std txt rate', 0.1054662887875882), ('fr...","['entry', 'std txt', 'txt rate', 'comp', 'win']","entry,std txt,txt rate,comp,win"


In [59]:
df_reject2 = df_reject.replace(r'^\s*$', np.nan, regex=True)

### Generating text based on default 1.3 bn parameter GPT Neo model

In [72]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [73]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [60]:
happy_gen_bn = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-1.3B")

07/04/2024 15:52:07 - INFO - happytransformer.happy_transformer -   Using device: mps


In [74]:
df_reject2["augm_text_default_bn_gpt_neo"]=df_reject2["yake_keywords_cutoff_ud"].apply(lambda x:gen_text_fn(happy_gen_bn,str(x)))


NotImplementedError: The operator 'aten::isin.Tensor_Tensor_out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

### Generating text based on default 125 mn parameter GPT Neo model

In [81]:
import torch
device = torch.device('cpu')  # or 'cuda' for GPU
happy_gen_mn = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M", device=device)

TypeError: __init__() got an unexpected keyword argument 'device'

In [78]:
happy_gen_mn = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")

07/04/2024 16:02:07 - INFO - happytransformer.happy_transformer -   Using device: mps


In [79]:
df_reject2["augm_text_default_mn_gpt_neo"]=df_reject2["yake_keywords_cutoff_ud"].apply(lambda x:gen_text_fn(happy_gen_mn,str(x)))


07/04/2024 16:02:16 - INFO - happytransformer.happy_transformer -   Moving model to mps
07/04/2024 16:02:16 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


NotImplementedError: The operator 'aten::isin.Tensor_Tensor_out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

### Generating text based on GPT Neo model with 1.3 bn parameters with recommended values of hyperparameters 

In [72]:
def gen_text_fn_arg(model,text_data,arg):
    result=model.generate_text(text_data,args=arg)
    return result.text

In [73]:
args_top_k = GENSettings(do_sample = True, top_k=5, no_repeat_ngram_size=3, early_stopping=True, min_length=1, max_length=50)


In [74]:
df_reject2["augm_text_bn_gpt_neo_arg"]=df_reject2["yake_keywords_cutoff_ud"].apply(lambda x:gen_text_fn_arg(happy_gen_bn,str(x),args_top_k))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

In [75]:
print(df_reject2.shape)
df_reject2.head(1)

(3757, 18)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,train_test_tag,label,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud,augm_text_default_bn_gpt_neo,augm_text_default_mn_gpt_neo,augm_text_bn_gpt_neo_arg
176,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,train,1,copies motion courthouse bayamón.,4,"[('copies motion courthouse', 0.01655915082773619), ('motion courthouse bayamón', 0.016559150827736194), ('copies motion', 0.04940384002065631), ('courthouse bayamón', 0.04940384002065631), ('motion courthouse', 0.09700399286574239), ('copies', 0.15831692877998726), ('bayamón', 0.15831692877998726), ('motion', 0.29736558256021506), ('courthouse', 0.29736558256021506)]","['motion', 'courthouse']","motion,courthouse",",courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,court",",and the court.\n\nThe court is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is",",courtroom,courtrooms,court-room,courts,courtenays,courthouses,courtens,courths,courtons,courtss,courtrs,courttens, courts,courtts, cour"


In [77]:
df_reject2.to_csv("sample_7K_inv_2019_aug_v1.csv",index=False)

### Generating text based on Fine tuned GPT Neo with 125 mn parameters on custom data of 100k samples

In [156]:
df_reject2=pd.read_csv("sample_7K_inv_2019_aug_v1.csv")
print(df_reject2.shape)
df_reject2.head(1)

(3757, 18)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,train_test_tag,label,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud,augm_text_default_bn_gpt_neo,augm_text_default_mn_gpt_neo,augm_text_bn_gpt_neo_arg
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,train,1,copies motion courthouse bayamón.,4,"[('copies motion courthouse', 0.01655915082773619), ('motion courthouse bayamón', 0.016559150827736194), ('copies motion', 0.04940384002065631), ('courthouse bayamón', 0.04940384002065631), ('motion courthouse', 0.09700399286574239), ('copies', 0.15831692877998726), ('bayamón', 0.15831692877998726), ('motion', 0.29736558256021506), ('courthouse', 0.29736558256021506)]","['motion', 'courthouse']","motion,courthouse",",courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,court",",and the court.\n\nThe court is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is",",courtroom,courtrooms,court-room,courts,courtenays,courthouses,courtens,courths,courtons,courtss,courtrs,courttens, courts,courtts, cour"


In [157]:
from happytransformer import GENTrainArgs

In [163]:
def gen_text_fn(model,text_data):
    result=model.generate_text(text_data)
    return result.text

In [159]:
print(df_train.shape)
df_train.head(1)

(73919, 16)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,train_test_tag,label,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud,aug_flag
0,2018-005511,878,16476175,2019-01-02 00:00:00 UTC,DDS INVOICE # 407908 CASE MANAGEMENT STATEMENT TO LASC CENTRAL,1.0,43.6,Approved,train,0,dds invoice # 407908 case management statement lasc central,9,"[('case management statement', 0.01655915082773619), ('statement lasc central', 0.016559150827736194), ('dds invoice', 0.02570861714399338), ('management statement lasc', 0.03339840940482845), ('case management', 0.04940384002065631), ('lasc central', 0.04940384002065631), ('management statement', 0.09700399286574239), ('statement lasc', 0.09700399286574239), ('dds', 0.15831692877998726), ('in...","['management', 'statement', 'lasc']","management,statement,lasc",default


In [82]:
df_train[["description"]].to_csv('sample_7K_inv_2019_description.txt', header=None, index=None, sep='\t', mode='a')


In [160]:
happy_gen_mn.train('sample_7K_inv_2019_description.txt')

03/16/2023 16:45:03 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 141.97it/s]
03/16/2023 16:45:04 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
/Users/d0s0ihi/opt/anaconda3/envs/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 588
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation step

Step,Training Loss
500,3.681600
1000,3.186000
1500,3.047100


Saving model checkpoint to /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-500
Configuration saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-500/config.json
Model weights saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-500/tokenizer_config.json
Special tokens file saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-1000
Configuration saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-1000/config.json
Model weights saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/tmp9380ryu6/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /var/folders/mz/k02t50s12ys220x346m8c24r0000gq

In [161]:
df_reject2.shape

(3757, 18)

In [164]:
df_reject2["augm_text_cstm_mn_gpt_neo"]=df_reject2["yake_keywords_cutoff_ud"].apply(lambda x:gen_text_fn(happy_gen_mn,str(x)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

In [166]:
df_reject2.to_csv("sample_7K_inv_2019_aug_v2.csv",index=False)

## Appending the aug data based on differenet methdology

In [167]:
## Reading the initial training data
df=pd.read_csv("sample_kw_7K_inv_2019.csv")
df2=df.copy()
df2["aug_flag"]="default"
df_train=df2[df2["train_test_tag"]=="train"]
print(df_train.shape)
df_train.head(1)

(73919, 16)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,train_test_tag,label,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud,aug_flag
0,2018-005511,878,16476175,2019-01-02 00:00:00 UTC,DDS INVOICE # 407908 CASE MANAGEMENT STATEMENT TO LASC CENTRAL,1.0,43.6,Approved,train,0,dds invoice # 407908 case management statement lasc central,9,"[('case management statement', 0.01655915082773619), ('statement lasc central', 0.016559150827736194), ('dds invoice', 0.02570861714399338), ('management statement lasc', 0.03339840940482845), ('case management', 0.04940384002065631), ('lasc central', 0.04940384002065631), ('management statement', 0.09700399286574239), ('statement lasc', 0.09700399286574239), ('dds', 0.15831692877998726), ('in...","['management', 'statement', 'lasc']","management,statement,lasc",default


In [168]:
req_columns=['matter_id',
 'invoice_id',
 'line_item_id',
 'line_item_date',
 'line_item_narative',
 'submitted_units',
 'submitted_rate',
 'Line_item_target',
 'description',
 'aug_flag']
new_columns=[
    'augm_text_default_bn_gpt_neo',
 'augm_text_default_mn_gpt_neo',
 'augm_text_bn_gpt_neo_arg',
#  'augm_text_cstm_mn_gpt_neo'
]
# req_columns

In [169]:
df_train2=df_train[req_columns]
print(df_train2.shape)
df_train2.head(1)

(73919, 10)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,description,aug_flag
0,2018-005511,878,16476175,2019-01-02 00:00:00 UTC,DDS INVOICE # 407908 CASE MANAGEMENT STATEMENT TO LASC CENTRAL,1.0,43.6,Approved,dds invoice # 407908 case management statement lasc central,default


In [170]:
## Reading the augmented data 
df_reject2=pd.read_csv("sample_7K_inv_2019_aug_v2.csv")
# df_reject2=pd.read_csv("sample_7K_inv_2019_aug_v1.csv")
df_reject2["aug_flag"]=np.nan
print(df_reject2.shape)
df_reject2.head(1)

(3757, 20)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,train_test_tag,label,description,word_count,yake_keywords,yake_keywords_cutoff,yake_keywords_cutoff_ud,augm_text_default_bn_gpt_neo,augm_text_default_mn_gpt_neo,augm_text_bn_gpt_neo_arg,augm_text_cstm_mn_gpt_neo,aug_flag
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,train,1,copies motion courthouse bayamón.,4,"[('copies motion courthouse', 0.01655915082773619), ('motion courthouse bayamón', 0.016559150827736194), ('copies motion', 0.04940384002065631), ('courthouse bayamón', 0.04940384002065631), ('motion courthouse', 0.09700399286574239), ('copies', 0.15831692877998726), ('bayamón', 0.15831692877998726), ('motion', 0.29736558256021506), ('courthouse', 0.29736558256021506)]","['motion', 'courthouse']","motion,courthouse",",courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,court",",and the court.\n\nThe court is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is",",courtroom,courtrooms,court-room,courts,courtenays,courthouses,courtens,courths,courtons,courtss,courtrs,courttens, courts,courtts, cour",", courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse",NaN


In [171]:
df_reject2[["yake_keywords_cutoff_ud"]].isnull().sum()

yake_keywords_cutoff_ud    1441
dtype: int64

In [172]:
## kepping the required columns
df_reject3=df_reject2[~(df_reject2["yake_keywords_cutoff_ud"].isnull())]
df_reject4=df_reject3[req_columns+new_columns]
print(df_reject4.shape)
df_reject4.head(1)

(2316, 13)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,description,aug_flag,augm_text_default_bn_gpt_neo,augm_text_default_mn_gpt_neo,augm_text_bn_gpt_neo_arg
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,copies motion courthouse bayamón.,NaN,",courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,court",",and the court.\n\nThe court is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is",",courtroom,courtrooms,court-room,courts,courtenays,courthouses,courtens,courths,courtons,courtss,courtrs,courttens, courts,courtts, cour"


## Append the aug data to initial training data

In [173]:
def aug_data_std_format(aug_data,text_column,aug_column):
    aug_data=aug_data[req_columns+[aug_column]]
    aug_data=aug_data.drop([text_column],axis=1)
    aug_data=aug_data.rename(columns={aug_column:text_column})
    aug_data["aug_flag"]=aug_column
    return aug_data

In [174]:
## Getting the aug data we got using default 1.3 bn parameter GPT Neo model
df_reject_default_bn_gpt_neo=aug_data_std_format(df_reject3,"description","augm_text_default_bn_gpt_neo")
print(df_reject_default_bn_gpt_neo.shape)
df_reject_default_bn_gpt_neo.head(1)

(2316, 10)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,aug_flag,description
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,augm_text_default_bn_gpt_neo,",courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,courtroom,court"


In [175]:
## Getting the aug data we got using default 125 mn parameter GPT Neo model
df_reject_default_mn_gpt_neo=aug_data_std_format(df_reject3,"description","augm_text_default_mn_gpt_neo")
print(df_reject_default_mn_gpt_neo.shape)
df_reject_default_mn_gpt_neo.head(1)

(2316, 10)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,aug_flag,description
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,augm_text_default_mn_gpt_neo,",and the court.\n\nThe court is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is held, and the court of appeals is the place where the court of appeals is"


In [176]:
## Getting the aug data we got using n GPT Neo model with 1.3 bn parameters with recommended values of hyperparameters
df_reject_default_bn_gpt_neo_arg=aug_data_std_format(df_reject3,"description","augm_text_bn_gpt_neo_arg")
print(df_reject_default_bn_gpt_neo_arg.shape)
df_reject_default_bn_gpt_neo_arg.head(1)


(2316, 10)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,aug_flag,description
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,augm_text_bn_gpt_neo_arg,",courtroom,courtrooms,court-room,courts,courtenays,courthouses,courtens,courths,courtons,courtss,courtrs,courttens, courts,courtts, cour"


In [177]:
## Getting the aug data we got using 125 mn parameter GPT Neo model tarined based on customised data
df_reject_custm_trn_default_mn_gpt_neo=aug_data_std_format(df_reject3,"description","augm_text_cstm_mn_gpt_neo")
print(df_reject_custm_trn_default_mn_gpt_neo.shape)
df_reject_custm_trn_default_mn_gpt_neo.head(1)


(2316, 10)


,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,aug_flag,description
0,2015-033056,30990,17222666,2019-04-09 00:00:00 UTC,copies of motion at the courthouse of Bayamón.,22.0,0.08,Rejected,augm_text_cstm_mn_gpt_neo,", courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse, courthouse"


## Appending all the aug data

In [180]:
df_aug=df_train2.append(df_reject_default_bn_gpt_neo).append(df_reject_default_mn_gpt_neo).append(df_reject_default_bn_gpt_neo_arg).append(df_reject_custm_trn_default_mn_gpt_neo)
# df_aug=df_train2.append(df_reject_default_bn_gpt_neo).append(df_reject_default_mn_gpt_neo).append(df_reject_default_bn_gpt_neo_arg)
print(df_aug.shape)
df_aug.head(1)


(83183, 10)


/var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/ipykernel_20316/346325670.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aug=df_train2.append(df_reject_default_bn_gpt_neo).append(df_reject_default_mn_gpt_neo).append(df_reject_default_bn_gpt_neo_arg).append(df_reject_custm_trn_default_mn_gpt_neo)
/var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/ipykernel_20316/346325670.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aug=df_train2.append(df_reject_default_bn_gpt_neo).append(df_reject_default_mn_gpt_neo).append(df_reject_default_bn_gpt_neo_arg).append(df_reject_custm_trn_default_mn_gpt_neo)
/var/folders/mz/k02t50s12ys220x346m8c24r0000gq/T/ipykernel_20316/346325670.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

,matter_id,invoice_id,line_item_id,line_item_date,line_item_narative,submitted_units,submitted_rate,Line_item_target,description,aug_flag
0,2018-005511,878,16476175,2019-01-02 00:00:00 UTC,DDS INVOICE # 407908 CASE MANAGEMENT STATEMENT TO LASC CENTRAL,1.0,43.6,Approved,dds invoice # 407908 case management statement lasc central,default


In [181]:
df_aug["aug_flag"].value_counts()

default                         73919
augm_text_default_bn_gpt_neo     2316
augm_text_default_mn_gpt_neo     2316
augm_text_bn_gpt_neo_arg         2316
augm_text_cstm_mn_gpt_neo        2316
Name: aug_flag, dtype: int64

In [182]:
pd.crosstab(df_aug["Line_item_target"],df_aug["aug_flag"])

aug_flag,augm_text_bn_gpt_neo_arg,augm_text_cstm_mn_gpt_neo,augm_text_default_bn_gpt_neo,augm_text_default_mn_gpt_neo,default
Line_item_target,,,,,
Approved,0,0,0,0,70162
Rejected,2316,2316,2316,2316,3757


In [183]:
df_aug.to_csv("sample_7K_inv_2019_aug_v3.csv",index=False)